In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-n3p91cnb
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-n3p91cnb
  Resolved https://github.com/Kaggle/learntools.git to commit ca2a51ee0085e4943cadaf5e9fe7dce2ec947d3c
  Preparing metadata (setup.py) ... done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268968 sha256=de842f232c26b91a45222aaa35b5978f4789ea7e63aad4c74f165d935b2fbf95
  Stored in directory: /tmp/pip-ephem-wheel-cache-oe175abi/wheels/2f/6c/3c/aa9f50cfb5a862157cb4c7a5b34881828cf45404698255dced
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.
Setup Complete


In [2]:
from google.cloud import bigquery
import pandas as pd

In [3]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,9f895c32eb7ef473c8c03da76e99a803dfcab41a,2130bc5fd239a4e3b304662424fb4cc7db0ca7abf78cc5...,2013-07-26 21:00:00+00:00,2013-07-26 21:00:00+00:00,60,0.00,NaN,NaN,NaN,NaN,...,NaN,NaN,Cash,None,NaN,NaN,None,NaN,NaN,None
1,f4c42db73e32ec8fd4814e5e22b4a1947d08c080,2130bc5fd239a4e3b304662424fb4cc7db0ca7abf78cc5...,2013-07-26 21:00:00+00:00,2013-07-26 21:00:00+00:00,0,0.00,NaN,NaN,NaN,NaN,...,NaN,NaN,Cash,None,NaN,NaN,None,NaN,NaN,None
2,3d147cb3f85483a1c00c7d7e27742712bf1e8f8c,38205a7b4cb9cdabd0d6ee8c4c6b115c53e7bd7b06645a...,2014-05-09 14:45:00+00:00,2014-05-09 14:45:00+00:00,120,0.00,NaN,NaN,NaN,NaN,...,NaN,NaN,Credit Card,None,NaN,NaN,None,NaN,NaN,None
3,522ac1df4121f3249d82761e1b37c91521f4e923,ac1045ae61a5226a9216bd9641b458bd9d7e7da0680ada...,2013-12-06 22:15:00+00:00,2013-12-06 22:30:00+00:00,840,0.35,NaN,NaN,NaN,NaN,...,NaN,NaN,Cash,None,NaN,NaN,None,NaN,NaN,None
4,93eb46600c3222a92d7d78cbac6ac35c42be9a77,78d45b51ff03d70f0ba2afda334acf93f678a179aaaaca...,2014-03-06 15:30:00+00:00,2014-03-06 15:45:00+00:00,1500,1.39,NaN,1.703198e+10,NaN,76.0,...,NaN,NaN,Credit Card,None,NaN,NaN,None,41.979071,-87.90304,POINT (-87.9030396611 41.9790708201)


## Comparison of the Mean, Median, and Standard Deviation of trip_seconds for Trips Made on Monday and Saturday

In [4]:
# Define the SQL query
query1 = """
WITH trips AS (
    SELECT
        trip_seconds,
        FORMAT_TIMESTAMP('%A', trip_start_timestamp) AS weekday
    FROM
        `bigquery-public-data.chicago_taxi_trips.taxi_trips`
)
SELECT
    weekday,
    AVG(trip_seconds) AS avg_seconds,
    APPROX_QUANTILES(trip_seconds, 100)[OFFSET(50)] AS median_seconds,
    STDDEV(trip_seconds) AS stddev_seconds
FROM
    trips
WHERE
    weekday IN ('Monday', 'Saturday')
GROUP BY
    weekday
"""

# Execute the query and convert results to a DataFrame
df1 = client.query(query1).to_dataframe()
df1.head()

,weekday,avg_seconds,median_seconds,stddev_seconds
0,Monday,839.479784,540,1346.060208
1,Saturday,737.410552,552,1146.335888


* The trip seconds on **Monday** had a **higher mean and standard deviation** than the trip duration on Saturday at 839 s (14 minutes) and 1346 s (22 minutes).
* The trip seconds on **Saturday** had a **higher median** than the trip duration on Monday at 552 s (9 minutes).

## The Five Routes with the Highest Number of Trips in 2023

In [5]:
# Define the SQL query
query2 = """
SELECT 
    pickup_community_area,
    dropoff_community_area,
    COUNT(*) AS num_trips
FROM 
    `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE 
    pickup_community_area IS NOT NULL
    AND dropoff_community_area IS NOT NULL
    AND EXTRACT(YEAR FROM trip_start_timestamp) = 2023
GROUP BY 
    pickup_community_area, dropoff_community_area
ORDER BY 
    num_trips DESC
LIMIT 5
"""

# Execute the query and convert results to a DataFrame
df2 = client.query(query2).to_dataframe()
df2.head(5)


,pickup_community_area,dropoff_community_area,num_trips
0,8,8,464844
1,32,8,291722
2,76,8,274747
3,8,32,267673
4,32,32,241596


**Community Area 8** is **the most** frequently used pickup and drop-off location in 2023 with a total of **464844 trips**.

## Comparison of Average Taxi Trip Costs (fare, tips, and taxes) by Payment Method in 2019

In [6]:
# Define the SQL query
query3 = """
SELECT 
    payment_type,
    AVG(fare) AS avg_fare,
    AVG(tips) AS avg_tips,
    AVG(tolls) AS avg_tolls
FROM 
    `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE 
    EXTRACT(YEAR FROM trip_start_timestamp) = 2019
GROUP BY 
    payment_type
ORDER BY 
    payment_type
"""

# Execute the query and convert results to a DataFrame
df3 = client.query(query3).to_dataframe()
df3.head(10)

,payment_type,avg_fare,avg_tips,avg_tolls
0,Cash,12.978306,0.002387,0.001791
1,Credit Card,16.814138,3.774545,0.002267
2,Dispute,15.655840,0.001446,0.078991
3,Mobile,15.973174,3.112698,0.000050
4,No Charge,15.753510,0.249532,0.019142
5,Pcard,11.327471,0.038023,0.000000
6,Prcard,16.130003,0.203528,0.001529
7,Prepaid,19.461415,0.000000,0.000000
8,Unknown,15.872380,0.082129,0.000147


* **Prepaid** has **the highest average fare** compared to other payment methods, but has **the lowest average tips and tolls** (no tips and tolls).
* **Credit card** payments have **the highest average tips** compared to other payment methods.
* **Mobile** payments have **the highest average tolls** compared to other payment methods.
* **Pcard** payments have **no tolls**.